<a href="https://colab.research.google.com/github/VincentBEDU/UrbanSim/blob/main/OD_scenarioB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 2  ScenarioB


Our estimates did sum to the grand total of flows, but this is because we were really fitting a 'total constrained' model which used $k$ - our constant of proportionality -  to ensure everything sort of added up (to within 1 commuter).

Where we have a full flow matrix to calibrate parameters, we can incorporate the row (origin) totals, column (destination) totals or both origin and destination totals to <i>constrain</i> our flow estimates to these known values.

There are various reasons for wanting to do this, for example:

1. If we are interested in flows of money into businesses or customers into shops, we might have information on the amount of disposable income and shopping habits of the people living in different areas from loyalty card data. This is known information about our origins and so we could constrain our spatial interaction model to this known information - we can make the assumption that this level of disposable income remains the same. We can then use other information about the attractiveness of places these people might like to shop in (store size, variety / specialism of goods etc.), to estimate how much money a new store opening in the area might make, or if a new out-of-town shopping centre opens, how much it might affect the business of shops in the town centre. This is what is known in the literature as the ‘retail model’ and is perhaps the most common example of a <b>Production (orign) Constrained Spatial Interaction Model</b>.

2. We might be interested in understanding the impact of a large new employer in an area on the flows of traffic in the vicinity or on the demand for new worker accommodation nearby. A good example of where this might be the case is with large new infrastructure developments like new airports. For example, before the go-ahead for the new third runway at Heathrow was given, one option being considered was a new runway in the Thames Estuary. If a new airport was built here, what would be the potential impact on transport flows in the area and where might workers commute from? This sort of scenario could be tested with an <b>Attraction (destination) Constrained Spatial Interaction Model</b> where the number of new jobs in a destination is known (as well as jobs in the surrounding area) and the model could be used to estimate where the workers will be drawn from (and their likely travel-to-work patterns).

3. We might be interested in understanding the changing patterns of commuting or migration over time. Data from the Census allows us to know an accurate snap-shot of migrating and commuting patterns every 10 years. In these full data matrices, we know both the numbers of commuters/migrants leaving origins and arriving at destinations as well as the interactions between them. If we constrain our model estimates to this known information at origin and destination, we can examine various things, including:
    - The ways that the patterns of commuting/migration differ from the model predictions - where we might get more migrant/commuter flows than we would expect.
    - How the model parameters vary over time - for example how does distance / cost of travel affect flows over time? Are people prepared to travel further or less far than before?

# IV Scenarios B

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import seaborn as sns
import folium
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats
import numpy as np
from math import sqrt

In [ ]:
# read London flow data
path="/content/drive/MyDrive/Dataset/london_flows.csv"
londflow = pd.read_csv(path)

In [ ]:
londflow.head()

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167


In [ ]:
# Assuming londflow is your DataFrame and 'distance' is the column name
# Remove rows where the 'distance' column has a value of 0
londflow1 = londflow[londflow['distance'] != 0]

# Now, londflow contains only the rows where distance is not zero

In [ ]:

# Filter rows where 'station_destination' is 'Canary Wharf'
canary_wharf_data = londflow1[londflow1['station_destination'] == 'Canary Wharf']

# Display the head of the filtered DataFrame along with the 'jobs' column
print(canary_wharf_data[['station_destination', 'jobs']].head())


    station_destination   jobs
3          Canary Wharf  58772
126        Canary Wharf  58772
348        Canary Wharf  58772
595        Canary Wharf  58772
817        Canary Wharf  58772


In [ ]:

# Reduce the number of jobs by 50% for rows where 'station_destination' is 'Canary Wharf'
#londflow1.loc[londflow1['station_destination'] == 'Canary Wharf', 'jobs'] *= 0.5


In [ ]:
# Filter rows where 'station_destination' is 'Canary Wharf'
canary_wharf_data = londflow1[londflow1['station_destination'] == 'Canary Wharf']

# Display the head of the filtered DataFrame along with the 'jobs' column
print(canary_wharf_data[['station_destination', 'jobs']].head())

    station_destination   jobs
3          Canary Wharf  58772
126        Canary Wharf  58772
348        Canary Wharf  58772
595        Canary Wharf  58772
817        Canary Wharf  58772


In [ ]:
londflow1.head()

,station_origin,station_destination,flows,population,jobs,distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097
1,Abbey Road,Beckton,1,599,442,8510.121774
2,Abbey Road,Blackwall,3,599,665,3775.448872
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220
4,Abbey Road,Canning Town,37,599,15428,2228.923167


In [ ]:
#set up the metric calculations
def CalcRSqaured(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2

    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)

    return RMSE

## Production Constrained Model

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

#take the variables and produce logarithms of them
x_variables = ["population", "jobs", "distance"]
log_x_vars = []
for x in x_variables:
    londflow1[f"log_{x}"] = np.log(londflow1[x])
    log_x_vars.append(f"log_{x}")

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-12-9e7bb6219314>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  londflow1[f"log_{x}"] = np.log(londflow1[x])
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-12-9e7bb6219314>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [ ]:
londflow1.head()

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance
0,Abbey Road,Bank and Monument,0,599,78549,8131.525097,6.395262,11.271478,9.003504
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274


In [ ]:
# Check for missing values
print(londflow1.isnull().sum())

# Check data types
print(londflow1.dtypes)

station_origin         0
station_destination    0
flows                  0
population             0
jobs                   0
distance               0
log_population         0
log_jobs               0
log_distance           0
dtype: int64
station_origin          object
station_destination     object
flows                    int64
population               int64
jobs                     int64
distance               float64
log_population         float64
log_jobs               float64
log_distance           float64
dtype: object


In [ ]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

try:
    # Filter rows where 'flows' is not zero
    filtered_londflow1 = londflow1[londflow1['flows'] != 0]

    # Create the formula (the "-1" indicates no intercept in the regression model).
    formula = 'flows ~ station_origin + log_jobs + log_distance - 1'

    # Run a production-constrained simulation with increased max iterations
    prodSim = smf.glm(formula=formula, data=filtered_londflow1, family=sm.families.Poisson()).fit(maxiter=1000)
    print(prodSim.summary())
except LinAlgError as e:
    print("Error:", e)


                 Generalized Linear Model Regression Results                  
Dep. Variable:                  flows   No. Observations:                43945
Model:                            GLM   Df Residuals:                    43545
Model Family:                 Poisson   Df Model:                          399
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -9.1409e+05
Date:                Tue, 09 Apr 2024   Deviance:                   1.6560e+06
Time:                        17:00:40   Pearson chi2:                 2.41e+06
No. Iterations:                     7   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                                                  coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------

So, what do we have?

Well, there are the elements of the model output that should be familiar from the unconstrained model:

The $\gamma$ parameter related to the destination attractiveness: 0.7302

The $\beta$ distance decay parameter: 0.8152. Recall the negative sign in the equation.

We can see from the standard outputs from the model that all of the explanatory variables are statistically significant (P>|z| < 0.01) and the z-scores indicate that the destination salary is having the most influence on the model, with distance following closely behind. And then we have a series of paramaters which are the vector of $\alpha_i$ values associated with our origin constraints.

## Model Estimates
Now at this point you will want to know what effect the constraints have had on the estimates produced by the model, so let's plug the parameters back into Equation 4 and look...


In [ ]:
#create some Oi and Dj columns in the dataframe and store row and column totals in them:
#to create O_i, take cdatasub ...then... group by origcodenew ...then... summarise by calculating the sum of Total
O_i = pd.DataFrame(filtered_londflow1.groupby(["station_origin"])["flows"].agg(np.sum))
O_i.rename(columns={"flows":"O_i"}, inplace = True)
filtered_londflow1 = filtered_londflow1.merge(O_i, on = "station_origin", how = "left" )

D_j = pd.DataFrame(filtered_londflow1.groupby(["station_destination"])["flows"].agg(np.sum))
D_j.rename(columns={"flows":"D_j"}, inplace = True)
filtered_londflow1 = filtered_londflow1.merge(D_j, on = "station_destination", how = "left" )

In [ ]:
filtered_londflow1.head()

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208


Now, fihing out the coeff. from the prodsim glm object

In [ ]:
#We can do this by pulling out the parameter values
coefs = pd.DataFrame(prodSim.params)
coefs.reset_index(inplace=True)
coefs.rename(columns = {0:"alpha_i", "index":"coef"}, inplace = True)
to_repl = ["(station_origin)", "\[", "\]"]
for x in to_repl:
    coefs["coef"] = coefs["coef"].str.replace(x, "",regex=True)
#then once you have done this you can join them back into the dataframes
filtered_londflow1 = filtered_londflow1.merge(coefs, left_on="station_origin", right_on="coef", how = "left")
filtered_londflow1.drop(columns = ["coef"], inplace = True)
#check this has worked
filtered_londflow1.head()

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442,3.270351
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665,3.270351
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772,3.270351
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428,3.270351
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208,3.270351


OK, now we can save our parameter values into some variables...

In [ ]:
# Creating a New variables for beta

alpha_i = prodSim.params[0:398]
gamma = prodSim.params[398]
beta = -prodSim.params[399]
#beta1 = 0.65           # not forgetting the negative in the formular
#beta2 = 0.75

In [ ]:
alpha_i

station_origin[Abbey Road]          3.270351
station_origin[Acton Central]       5.008886
station_origin[Acton Town]          4.397394
station_origin[Aldgate]             3.361125
station_origin[Aldgate East]        3.408728
                                      ...   
station_origin[Wood Street]         5.672160
station_origin[Woodford]            4.955425
station_origin[Woodgrange Park]     5.320215
station_origin[Woodside Park]       4.496709
station_origin[Woolwich Arsenal]    6.701868
Length: 398, dtype: float64

In [ ]:
gamma

0.7301699265793882

In [ ]:
beta

0.8151874614788027

Ready to generate some estimates

In [ ]:
filtered_londflow1["prodsimest1"] = np.exp(filtered_londflow1["alpha_i"]+gamma*filtered_londflow1["log_jobs"]
                                 - beta*filtered_londflow1["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
filtered_londflow1.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i,prodsimest1
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442,3.270351,1.406918
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665,3.270351,3.677350
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772,3.270351,76.062924
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428,3.270351,56.123931
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208,3.270351,3.568414
5,Abbey Road,Cutty Sark,2,599,1748,8503.898909,6.395262,7.466228,9.048280,599,1748,3.270351,3.841726
6,Abbey Road,Cyprus,7,599,850,6532.099618,6.395262,6.745236,8.784484,599,850,3.270351,2.813752
7,Abbey Road,Devons Road,1,599,611,3958.324171,6.395262,6.415097,8.283576,599,611,3.270351,3.326081
8,Abbey Road,East India,2,599,1522,3384.141666,6.395262,7.327781,8.126856,599,1522,3.270351,7.359304
9,Abbey Road,Island Gardens,2,599,691,7706.296370,6.395262,6.538140,8.949793,599,691,3.270351,2.113923


## Assessing the model output

So what do the outputs from our Production Constrained Model look like? How has the goodness-of-fit improved and how can we start to use this a bit like a retail model and assess the likley impacts of changing detsination attractiveness etc.?

### The flow matrics

In [ ]:
#first round the estimates
#filtered_londflow1["prodsimest1"] = round(filtered_londflow1["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
londflowmat3 = filtered_londflow1.pivot_table(values ="prodsimest1", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)

londflowmat3.fillna(0, inplace=True)

londflowmat3

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.080449,599.0
Acton Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1224.0
Acton Town,0.000000,0.000000,0.000000,20.113147,20.161026,0.000000,9.812411,0.000000,0.000000,21.971277,...,18.345025,3.593363,6.111567,13.966017,0.000000,0.000000,0.000000,0.000000,0.000000,3745.0
Aldgate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.397174,...,0.000000,0.000000,3.537277,2.867179,0.000000,0.000000,0.000000,0.000000,0.000000,2886.0
Aldgate East,0.000000,0.000000,2.942747,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.217503,...,5.759102,0.000000,0.000000,2.959688,0.000000,1.484809,0.000000,1.308642,0.000000,3172.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.000000,0.000000,8.426657,37.212112,40.784321,0.000000,0.000000,0.000000,0.000000,34.473399,...,17.200010,0.000000,10.962554,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4868.0
Woodgrange Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,530.0
Woodside Park,0.000000,0.000000,6.211346,21.790171,21.846134,0.000000,0.000000,0.000000,0.000000,26.202682,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3093.0


## Ready to generate the new  estimate with beta = 0.65


Error: 'alpha_i' column does not exist in the DataFrame.


,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance
1,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012
2,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275
3,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348
4,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274
5,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842
7,Abbey Road,Cutty Sark,2,599,1748,8503.898909,6.395262,7.466228,9.048280
8,Abbey Road,Cyprus,7,599,850,6532.099618,6.395262,6.745236,8.784484
9,Abbey Road,Devons Road,1,599,611,3958.324171,6.395262,6.415097,8.283576
10,Abbey Road,East India,2,599,1522,3384.141666,6.395262,7.327781,8.126856
11,Abbey Road,Island Gardens,2,599,691,7706.296370,6.395262,6.538140,8.949793


Here it is very easy to see the Origin Constrained working. The sum across all destinations for each origin in the estimated matrix is exactly the same sum (give or take 1 or 2) across the observed matrics - $\sum_j T_{ij} = \sum_j \lambda_{ij} = O_i$, but clearly, the same is not true when you sum across all origins for each destination - $\sum_i T_{ij} \neq \sum_i \lambda_{ij} \neq D_j$

### 2.2.2 How do the fits compare with the unconstrained model from last time?

In [ ]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2

    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)

    return RMSE

In [ ]:
CalcRSquared(filtered_londflow1["flows"], filtered_londflow1["prodsimest1"])

0.3937304850830748

In [ ]:
CalcRMSE(filtered_londflow1["flows"], filtered_londflow1["prodsimest1"])

120.146

Clearly by constraining our model estimates to known origin totals, the fit of the model has improved quite considerably - from around 0.3114 in the unconstrained model to around 0.3937 in this model. The RMSE has also dropped quite noticeably (from 128.178 to 120.146)

# Scenario B

To reflect the significant increase in the cost of transport (distance) the actual beta has to be reduced. The original or actual beta = -0.8151874614788027. Therefore, beta = -0.65 and beta = -0.75 will be adopted.

In [ ]:
# Creating a New variables for beta

alpha_i = prodSim.params[0:398]
gamma = prodSim.params[398]
#beta = -prodSim.params[399]
beta1 = 0.65           # not forgetting the negative in the formular
beta2 = 0.75

In [ ]:
alpha_i

station_origin[Abbey Road]          3.270351
station_origin[Acton Central]       5.008886
station_origin[Acton Town]          4.397394
station_origin[Aldgate]             3.361125
station_origin[Aldgate East]        3.408728
                                      ...   
station_origin[Wood Street]         5.672160
station_origin[Woodford]            4.955425
station_origin[Woodgrange Park]     5.320215
station_origin[Woodside Park]       4.496709
station_origin[Woolwich Arsenal]    6.701868
Length: 398, dtype: float64

In [ ]:
gamma

0.7301699265793882

In [ ]:
beta1

0.65

## Ready to generate the new  estimate with beta = 0.65

In [ ]:
filtered_londflow1["prodsimest2"] = np.exp(filtered_londflow1["alpha_i"]+gamma*filtered_londflow1["log_jobs"]
                                 - beta1*filtered_londflow1["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
filtered_londflow1.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i,prodsimest1,prodsimest2
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442,3.270351,1.406918,6.272563
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665,3.270351,3.677350,14.335254
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772,3.270351,76.062924,311.477852
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428,3.270351,56.123931,200.544803
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208,3.270351,3.568414,15.287971
5,Abbey Road,Cutty Sark,2,599,1748,8503.898909,6.395262,7.466228,9.048280,599,1748,3.270351,3.841726,17.125765
6,Abbey Road,Cyprus,7,599,850,6532.099618,6.395262,6.745236,8.784484,599,850,3.270351,2.813752,12.008390
7,Abbey Road,Devons Road,1,599,611,3958.324171,6.395262,6.415097,8.283576,599,611,3.270351,3.326081,13.067623
8,Abbey Road,East India,2,599,1522,3384.141666,6.395262,7.327781,8.126856,599,1522,3.270351,7.359304,28.174578
9,Abbey Road,Island Gardens,2,599,691,7706.296370,6.395262,6.538140,8.949793,599,691,3.270351,2.113923,9.271441


## Assessing the Output

Flow Metrics

In [ ]:
#first round the estimates
#filtered_londflow1["prodsimest1"] = round(filtered_londflow1["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
londflowmat4 = filtered_londflow1.pivot_table(values ="prodsimest2", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)

londflowmat4.fillna(0, inplace=True)

londflowmat4

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,31.996581,2.076179e+03
Acton Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.434237e+03
Acton Town,0.000000,0.000000,0.000000,100.475609,100.951043,0.000000,40.032438,0.000000,0.00000,108.456862,...,90.483678,17.400034,32.120723,59.322401,0.000000,0.000000,0.000000,0.000000,0.000000,1.746521e+04
Aldgate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,101.768265,...,0.000000,0.000000,16.835602,13.606355,0.000000,0.000000,0.000000,0.000000,0.000000,1.077368e+04
Aldgate East,0.000000,0.000000,14.735035,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,102.196634,...,29.400828,0.000000,0.000000,14.090539,0.000000,7.217301,0.000000,6.578012,0.000000,1.219677e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.000000,0.000000,46.642738,183.750489,198.242812,0.000000,0.000000,0.000000,0.00000,173.921502,...,96.241309,0.000000,57.310279,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.432339e+04
Woodgrange Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.202215e+03
Woodside Park,0.000000,0.000000,33.326421,109.278192,109.811678,0.000000,0.000000,0.000000,0.00000,127.346716,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.530278e+04


## Goodness of Fit of the Model

In [ ]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2

    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)

    return RMSE

In [ ]:
CalcRSquared(filtered_londflow1["flows"], filtered_londflow1["prodsimest2"])

0.4182730072237843

In [ ]:
CalcRMSE(filtered_londflow1["flows"], filtered_londflow1["prodsimest2"])

315.286

## The second value of beta  

In [ ]:
# Creating a New variables for beta

alpha_i = prodSim.params[0:398]
gamma = prodSim.params[398]
#beta = -prodSim.params[399]
beta1 = 0.65           # not forgetting the negative in the formular
beta2 = 0.75

In [ ]:
alpha_i

station_origin[Abbey Road]          3.270351
station_origin[Acton Central]       5.008886
station_origin[Acton Town]          4.397394
station_origin[Aldgate]             3.361125
station_origin[Aldgate East]        3.408728
                                      ...   
station_origin[Wood Street]         5.672160
station_origin[Woodford]            4.955425
station_origin[Woodgrange Park]     5.320215
station_origin[Woodside Park]       4.496709
station_origin[Woolwich Arsenal]    6.701868
Length: 398, dtype: float64

In [ ]:
gamma

0.7301699265793882

In [ ]:
beta2

0.75

## Ready to generate the new  estimate with beta = 0.75

In [ ]:
filtered_londflow1["prodsimest3"] = np.exp(filtered_londflow1["alpha_i"]+gamma*filtered_londflow1["log_jobs"]
                                 - beta2*filtered_londflow1["log_distance"])
#or you could do it the easy way like we did last week with the fitted column (See previous practical)
filtered_londflow1.head(10)

,station_origin,station_destination,flows,population,jobs,distance,log_population,log_jobs,log_distance,O_i,D_j,alpha_i,prodsimest1,prodsimest2,prodsimest3
0,Abbey Road,Beckton,1,599,442,8510.121774,6.395262,6.091310,9.049012,599,442,3.270351,1.406918,6.272563,2.537765
1,Abbey Road,Blackwall,3,599,665,3775.448872,6.395262,6.499787,8.236275,599,665,3.270351,3.677350,14.335254,6.290838
2,Abbey Road,Canary Wharf,1,599,58772,5086.514220,6.395262,10.981421,8.534348,599,58772,3.270351,76.062924,311.477852,132.673793
3,Abbey Road,Canning Town,37,599,15428,2228.923167,6.395262,9.643939,7.709274,599,15428,3.270351,56.123931,200.544803,92.768792
4,Abbey Road,Crossharbour,1,599,1208,6686.475560,6.395262,7.096721,8.807842,599,1208,3.270351,3.568414,15.287971,6.336218
5,Abbey Road,Cutty Sark,2,599,1748,8503.898909,6.395262,7.466228,9.048280,599,1748,3.270351,3.841726,17.125765,6.929281
6,Abbey Road,Cyprus,7,599,850,6532.099618,6.395262,6.745236,8.784484,599,850,3.270351,2.813752,12.008390,4.988609
7,Abbey Road,Devons Road,1,599,611,3958.324171,6.395262,6.415097,8.283576,599,611,3.270351,3.326081,13.067623,5.707494
8,Abbey Road,East India,2,599,1522,3384.141666,6.395262,7.327781,8.126856,599,1522,3.270351,7.359304,28.174578,12.500073
9,Abbey Road,Island Gardens,2,599,691,7706.296370,6.395262,6.538140,8.949793,599,691,3.270351,2.113923,9.271441,3.788460


## Assessing the Output

Flow Matrics

In [ ]:
#first round the estimates
#filtered_londflow1["prodsimest1"] = round(filtered_londflow1["prodsimest1"],0)
#now we can create a pivot tabel to turn the paired list into a matrix, and compute the margins as well
londflowmat5 = filtered_londflow1.pivot_table(values ="prodsimest3", index="station_origin", columns = "station_destination",
                            aggfunc=np.sum, margins=True)

londflowmat5.fillna(0, inplace=True)

londflowmat5

station_destination,Abbey Road,Acton Central,Acton Town,Aldgate,Aldgate East,All Saints,Alperton,Amersham,Anerley,Angel,...,Wimbledon,Wimbledon Park,Wood Green,Wood Lane,Wood Street,Woodford,Woodgrange Park,Woodside Park,Woolwich Arsenal,All
station_origin,,,,,,,,,,,,,,,,,,,,,
Abbey Road,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,12.839807,9.760444e+02
Acton Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.198740e+03
Acton Town,0.000000,0.000000,0.000000,37.945381,38.070894,0.000000,17.090290,0.000000,0.000000,41.256318,...,34.436217,6.696379,11.763426,24.714744,0.000000,0.000000,0.000000,0.000000,0.000000,6.868000e+03
Aldgate,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.960449,...,0.000000,0.000000,6.547153,5.300734,0.000000,0.000000,0.000000,0.000000,0.000000,4.840131e+03
Aldgate East,0.000000,0.000000,5.556911,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,44.849224,...,10.958540,0.000000,0.000000,5.478700,0.000000,2.771167,0.000000,2.474928,0.000000,5.384710e+03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodford,0.000000,0.000000,16.554425,69.883669,76.117651,0.000000,0.000000,0.000000,0.000000,65.291310,...,33.934688,0.000000,21.056231,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.178932e+03
Woodgrange Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.275660e+02
Woodside Park,0.000000,0.000000,12.053325,41.172510,41.315817,0.000000,0.000000,0.000000,0.000000,48.900505,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.810015e+03


## Goodness of fit of the model

In [ ]:
def CalcRSquared(observed, estimated):
    """Calculate the r^2 from a series of observed and estimated target values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    r, p = scipy.stats.pearsonr(observed, estimated)
    R2 = r **2

    return R2

def CalcRMSE(observed, estimated):
    """Calculate Root Mean Square Error between a series of observed and estimated values
    inputs:
    Observed: Series of actual observed values
    estimated: Series of predicted values"""

    res = (observed -estimated)**2
    RMSE = round(sqrt(res.mean()), 3)

    return RMSE

In [ ]:
CalcRSquared(filtered_londflow1["flows"], filtered_londflow1["prodsimest3"])

0.40727515084031013

In [ ]:
CalcRMSE(filtered_londflow1["flows"], filtered_londflow1["prodsimest3"])

137.334

## Chceking the three Prodsim estinmates

In [ ]:
#check
filtered_londflow1[["prodsimest1", "prodsimest2", "prodsimest3"]]

,prodsimest1,prodsimest2,prodsimest3
0,1.406918,6.272563,2.537765
1,3.677350,14.335254,6.290838
2,76.062924,311.477852,132.673793
3,56.123931,200.544803,92.768792
4,3.568414,15.287971,6.336218
...,...,...,...
43940,131.612217,632.487260,244.531878
43941,268.799374,1202.816647,485.557029
43942,36.861579,167.463732,66.985509
43943,106.060284,438.026544,185.619016


In [ ]:
#check
print(filtered_londflow1[["prodsimest1", "prodsimest2", "prodsimest3"]].head(50))

    prodsimest1  prodsimest2  prodsimest3
0      1.406918     6.272563     2.537765
1      3.677350    14.335254     6.290838
2     76.062924   311.477852   132.673793
3     56.123931   200.544803    92.768792
4      3.568414    15.287971     6.336218
5      3.841726    17.125765     6.929281
6      2.813752    12.008390     4.988609
7      3.326081    13.067623     5.707494
8      7.359304    28.174578    12.500073
9      2.113923     9.271441     3.788460
10     2.603262    11.185718     4.627799
11     6.716939    27.253709    11.673606
12     7.361840    31.111001    13.001529
13     5.683201    23.767561     9.995650
14     4.560261    18.390439     7.906365
15     3.591437    14.331451     6.200801
16     2.874532    11.828329     5.023528
17    13.637599    58.277359    24.190994
18    10.208297    43.064701    18.016136
19    11.556615    38.229887    18.529684
20   242.387312   779.133371   384.260800
21    17.352939    50.652855    26.482915
22     9.541767    34.737530    15